# How to run a simulation with CoMETS

CoMETS offers a specific interface to manipulate Cosmo Tech's simulators: the `CosmoInterface`. We will see in this tutorial how to use it and perform basic operations like running a simulation, setting specific inputs or getting the simulation results. 

This tutorial assumes you already have some level of familiarity with Cosmo models.

## The Brewery model

We are working with a drink consumption model including stock level management and customer satisfaction dynamics. The parameters we will use to change the model's behavior are: 

- `NbWaiters`: number of waiters in the bar
- `RestockQty`: a fixed quantity of drinks added to the current stock of the bar once it reaches a threshold value.

The model contains two probes:

- `Stock`: stock of the bar at each time step
- `Satisfaction`: satisfaction of each client at each time step

The model also has two consumers (one for each probe), storing the result of each probe in a CSV file (CSVFileGenericConsumer). 

## Importing the library
In order to use the CosmoInterface to manipulate our simulator we first need to import CoMETS

In [1]:
import comets

## Creating the `CosmoInterface`
We can now create our `CosmoInterface`. It will be declared with three arguments:

 - `simulator_path` (mandatory): path to the simulation file, usually inside the project folder 'Simulation'
 
 - `project_path` (optional): path to the project containing the model. This should be an **absolute path**. If not specified, assumes we are running from inside the project folder    
    
 - `temporary_consumers` (optional): boolean to specify if the CSVFileGenericConsumers are written in a temporary folder accessible by the CosmoInterface. By default is equal to False, so the consumers are written in their original location (in the folder 'Simulation/Output/') and the CosmoInterface has no access to the consumers.

In [2]:
#This example assumes that this notebook is in the folder containing the Brewery project
#and that the model has been compiled with the python wrappers activated.

from pathlib import Path
cwd = Path().resolve()

simulator = comets.CosmoInterface(
    simulator_path = 'BreweryTutorialSimulation',
    project_path = cwd,
    temporary_consumers = True
)

Now that the `CosmoInterface` is created, we can start to manipulate the simulator. 

## Initializing the simulator
To interact with the Cosmo Tech simulator we first need to initialize it, which will load the instance and its associated data:

In [3]:
simulator.initialize()

## Setting parameters
Now that the simulator is loaded, we have the possibility to change the value of an attribute in the model. For that we use the `set_inputs()`  method of `CosmoInterface` class, which takes as argument a `ParameterSet`: a set of parameters, stored as a dictionary containing names of parameters and their values. For a Cosmo Tech model, the name of a parameter corresponds to it localization in the model, given as a datapath. For instance NBWaiters and RestockQty datapath are:
 - `Model::{Entity}MyBar::@NbWaiters`
 - `Model::{Entity}MyBar::@RestockQty`

For example, the correct ParameterSet to set the value of `NbWaiters` to 10 and `RestockQty` to 12 is: 

$$
\text{parameter } = \text{{'Model::{Entity}MyBar::@NbWaiters': 10, 'Model::{Entity}MyBar::@RestockQty': 12}}
$$

In [4]:
parameter = {'Model::{Entity}MyBar::@NbWaiters': 10,
             'Model::{Entity}MyBar::@RestockQty': 12}
simulator.set_inputs(parameter)

## Running the simulation
To run the simulation, we simply use the `run()` method.

In [5]:
simulator.run()

## Getting the simulation's results
Now that the simulation is run, we can use the `get_outputs()` method of `CosmoInterface` class to get the final value of specific attributes in the model. This method takes a list of the output parameters' name as an argument. Remember, in a Cosmo Tech model, the name of a parameter corresponds to its datapath. For instance, the Stock datapath is: `Model::{Entity}MyBar::@Stock`

Note that the output format of the `get_outputs()` method is a `ParameterSet`.

In [6]:
list_of_output_parameters = ['Model::{Entity}MyBar::@Stock']
simulator.get_outputs(list_of_output_parameters)

{'Model::{Entity}MyBar::@Stock': 17}

## Getting the simulation's consumers
As we set the argument `temporary_consumers` to True when we created the CosmoInterface, we can get the simulation's consumers. This simulation (i.e 'BreweryTutorialSimulation') contains two consumers: one CSV file for the probe on the Stock and one CSV file for the probe on customer Satisfaction. To get the consumers we use the `get_consumers()` method. It returns a list of pandas dataframes corresponding to each file generated by a CSVFileGenericConsumer during the simulation. Note that this method requires a valid installation of pandas.

In [11]:
for dataframes in simulator.get_consumers():
    display (dataframes.head())

,iteration,Entity name,Attribute name,Attribute value
0,0,MyBar,Stock,10
1,1,MyBar,Stock,8
2,2,MyBar,Stock,6
3,3,MyBar,Stock,15
4,4,MyBar,Stock,12


,iteration,Entity name,Attribute name,Attribute value
0,0,Customer1,Satisfaction,0
1,1,Customer1,Satisfaction,1
2,2,Customer1,Satisfaction,1
3,3,Customer1,Satisfaction,1
4,4,Customer1,Satisfaction,1


## Getting the simulation's results with the consumers
In this simulation, we consider that the stock at the end of the simulation is an output. To retrieve it, we previously used the `get_outputs()` method. However, this information is also available in the consumer that stored the StockProbe result in a CSV file. More precisely, as we want the stock value at the end of the simulation, this corresponds to the last probe result: the last row of the CSV file. We can get it with the following command: 

In [12]:
simulator.get_consumers()[0].iloc[-1]

iteration             53
Entity name        MyBar
Attribute name     Stock
Attribute value       17
Name: 53, dtype: object

## Destroying the simulator
Once we have finished to use the CosmoInterface and manipulate the simulator, we can terminate it, which will also clean up the temporary directory where consumers are written (note that you can get the path of this directory using `simulator.get_consumers_directory()`).

In [13]:
simulator.terminate()

Now that you know how to use CoMETS to manipulate simulators, you will see in the next tutorial how use the CosmoInterface to create a Task.